In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import TimeSeriesSplit


In [3]:
data = pd.read_excel("../../Data/Testdata_Models/AirQualityUCI.xlsx")

data["datetime"] = pd.to_datetime(data["Date"].astype(str) + " " + data["Time"].astype(str))
data.drop(columns=['PT08.S1(CO)', 'NMHC(GT)', 'C6H6(GT)', 'PT08.S2(NMHC)', 'NOx(GT)', 'PT08.S3(NOx)', 'NO2(GT)', 'PT08.S4(NO2)', 'PT08.S5(O3)'], inplace=True)

data = data.sort_values("datetime").reset_index(drop=True)
data = data[["datetime", "CO(GT)", "T", "RH", "AH"]].rename(columns={
    "CO(GT)": "Y",
})

data = data.replace(-200, np.nan)

In [3]:
# range of values
df = data.copy()
print(f"Min: {df['datetime'].min()} Max: {df['datetime'].max()}")

Min: 2004-03-10 18:00:00 Max: 2005-04-04 14:00:00


In [4]:
# How many varibales are nan
print(f"Shape before dropping Nans: {df.shape}")
df = df.dropna()
print(f"Shape after dropping Nans: {df.shape}")

Shape before dropping Nans: (9357, 5)
Shape after dropping Nans: (7344, 5)


In [5]:
df = df.set_index("datetime")


In [6]:
# --- build AR lags and drop NaNs (keep only Y_lag* as features) ---
def make_ar_lags(df, target="Y", lags=72):
    out = df[[target]].copy()
    for k in range(1, lags+1):
        out[f"{target}_lag{k}"] = out[target].shift(k)
    return out.dropna()

lags = 144  # use as much history as you want (e.g., 3 days if hourly)
ar = make_ar_lags(df, "Y", lags=lags)

# split
train = ar.loc[ar.index < "2004-11-01"]
test  = ar.loc[ar.index >= "2004-11-01"]

lag_cols = [c for c in train.columns if c.startswith("Y_lag")]
X_train, y_train = train[lag_cols], train["Y"]
X_test,  y_test  = test[lag_cols],  test["Y"]

# model
model = LinearRegression().fit(X_train, y_train)
print(X_test.shape)


/var/folders/q9/d545ygdx5rj6lg_00gpykppw0000gn/T/ipykernel_64543/4117726918.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[f"{target}_lag{k}"] = out[target].shift(k)
/var/folders/q9/d545ygdx5rj6lg_00gpykppw0000gn/T/ipykernel_64543/4117726918.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[f"{target}_lag{k}"] = out[target].shift(k)
/var/folders/q9/d545ygdx5rj6lg_00gpykppw0000gn/T/ipykernel_64543/4117726918.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.inse

(3252, 144)


In [7]:
# # --- recursive k-step forecast helper (pure AR) ---
# def recursive_ar_forecast(model, lag_vector, lag_cols, steps):
#     # coerce to 1D numpy array
#     if isinstance(lag_vector, pd.Series):
#         lag_vals = lag_vector.values
#     else:
#         lag_vals = np.asarray(lag_vector)

#     preds = []
#     for _ in range(steps):
#         X = pd.DataFrame([lag_vals], columns=lag_cols)  # keep feature names
#         y_hat = model.predict(X)[0]
#         preds.append(y_hat)
#         # shift: insert prediction at front, drop last lag
#         lag_vals = np.concatenate(([y_hat], lag_vals[:-1]))
#     return np.array(preds)


# block_len = 5  # number of rows per forecast block (set to 3 if you want 3-step horizon)

# predictions, truths, timestamps = [], [], []
# lag_cols = list(X_train.columns)

# i = 0
# test_idx = X_test.index
# while i < len(test_idx):
#     print(i)
#     idx_block = test_idx[i : i + block_len]            # exactly last 5 rows of this chunk
#     if len(idx_block) == 0:
#         break

#     # initial lags for the first timestamp in this block
#     lag_vec = X_test.loc[idx_block[0], lag_cols]       # Series with names

#     # recursive 5-step (len(idx_block)) forecast
#     preds_block = recursive_ar_forecast(
#         model=model,
#         lag_vector=lag_vec,
#         lag_cols=lag_cols,
#         steps=len(idx_block)
#     )

#     predictions.extend(preds_block)
#     truths.extend(y_test.loc[idx_block].to_numpy())
#     timestamps.extend(idx_block.tolist())

#     i += block_len    
# results = pd.DataFrame({"datetime": timestamps, "y_true": truths, "y_pred": predictions}).set_index("datetime")
# rmse = mean_squared_error(results["y_true"], results["y_pred"])  # RMSE
# print(f"RMSE (5-step recursive AR): {rmse:.3f}")

### With covariates

In [8]:


# # --- 1) Build lag features for Y and covariates ---
# def make_lagged_features(df, target="Y", covariates=("T","RH","AH"), lags=72):
#     # Build a single list of Series/DataFrames, then concat once (no repeated inserts)
#     parts = [df[[target]]]  # keep original target (for y)
#     # target lags
#     parts += [df[target].shift(k).rename(f"{target}_lag{k}") for k in range(1, lags + 1)]
#     # covariate lags
#     for c in covariates:
#         parts += [df[c].shift(k).rename(f"{c}_lag{k}") for k in range(1, lags + 1)]
#     out = pd.concat(parts, axis=1)
#     return out.dropna()

# lags = 72           # history length (keep as you like)
# block_len = 5       # <- your “last 5 entries” per block (set to 3 if you want 3-step)

# lagged = make_lagged_features(df, target="Y", covariates=("T","RH","AH"), lags=lags)

# # --- 2) Train/test split (November start) ---
# train = lagged.loc[lagged.index < "2004-11-01"]
# test  = lagged.loc[lagged.index >= "2004-11-01"]

# y_col = "Y"
# y_lag_cols  = [f"Y_lag{k}" for k in range(1, lags+1)]
# cov_lag_cols = [c for c in lagged.columns if c.startswith("T_lag") or c.startswith("RH_lag") or c.startswith("AH_lag")]
# feature_cols = y_lag_cols + cov_lag_cols  # keep a fixed, known order

# X_train, y_train = train[feature_cols], train[y_col]
# X_test,  y_test  = test[feature_cols],  test[y_col]

# # --- 3) Fit linear baseline ---
# model = LinearRegression().fit(X_train, y_train)

# # --- 4) Recursive multi-step forecast in fixed-size row blocks (no time windowing) ---
# predictions, truths, timestamps = [], [], []
# test_idx = X_test.index
# i = 0

# while i < len(test_idx):
#     idx_block = test_idx[i : i + block_len]
#     if len(idx_block) == 0:
#         break

#     # initialize Y lags from the first row of the block
#     y_lags = X_test.loc[idx_block[0], y_lag_cols].to_numpy()

#     # step through the block recursively
#     for ts in idx_block:
#         row = X_test.loc[ts, feature_cols].copy()  # covariate lags for this timestamp
#         # overwrite Y lag columns with our *current* y_lags (so we don't use future truth)
#         row.loc[y_lag_cols] = y_lags
#         y_hat = model.predict(pd.DataFrame([row], columns=feature_cols))[0]

#         predictions.append(y_hat)
#         truths.append(y_test.loc[ts])
#         timestamps.append(ts)

#         # shift in our prediction for the next step
#         y_lags = np.concatenate(([y_hat], y_lags[:-1]))

#     i += block_len

# results = pd.DataFrame(
#     {"datetime": timestamps, "y_true": truths, "y_pred": predictions}
# ).set_index("datetime")

# rmse = mean_squared_error(results["y_true"], results["y_pred"])
# print(f"RMSE (recursive with Y + covariate lags): {rmse:.3f}")


In [9]:
# tscv = TimeSeriesSplit(n_splits=5)
# model = Pipeline([
#     ("scaler", StandardScaler()),
#     ("reg", RidgeCV(alphas=np.logspace(-4, 3, 60), cv=tscv))
# ])
# model.fit(X_train, y_train)
# predictions, truths, timestamps = [], [], []
# test_idx = X_test.index
# i = 0


# while i < len(test_idx):
#     idx_block = test_idx[i : i + block_len]
#     if len(idx_block) == 0:
#         break

#     # initialize Y lags from the first row of the block
#     y_lags = X_test.loc[idx_block[0], y_lag_cols].to_numpy()
#     print(i)
#     # step through the block recursively
#     for ts in idx_block:
#         row = X_test.loc[ts, feature_cols].copy()  # covariate lags for this timestamp
#         # overwrite Y lag columns with our *current* y_lags (so we don't use future truth)
#         row.loc[y_lag_cols] = y_lags
#         y_hat = model.predict(pd.DataFrame([row], columns=feature_cols))[0]

#         predictions.append(y_hat)
#         truths.append(y_test.loc[ts])
#         timestamps.append(ts)

#         # shift in our prediction for the next step
#         y_lags = np.concatenate(([y_hat], y_lags[:-1]))

#     i += block_len

# results = pd.DataFrame(
#     {"datetime": timestamps, "y_true": truths, "y_pred": predictions}
# ).set_index("datetime")

# rmse = mean_squared_error(results["y_true"], results["y_pred"])
# print(f"RMSE (recursive with Y + covariate lags): {rmse:.3f}")


In [10]:
# # --- TimesFM: batched block forecasts on your series Y ---
# import torch
# import numpy as np
# import pandas as pd
# from sklearn.metrics import mean_squared_error
# import timesfm

# # 0) Model (GPU if available)
# torch.set_float32_matmul_precision("high")
# if torch.backends.mps.is_available():
#     device = "mps"  # Metal Performance Shaders (for Apple Silicon)
# elif torch.cuda.is_available():
#     device = "cuda"
# else:
#     device = "cpu"

# model = timesfm.TimesFM_2p5_200M_torch.from_pretrained("google/timesfm-2.5-200m-pytorch")
# cfg = timesfm.ForecastConfig(
#     max_context=1024,         # keep last 1024 points of history per block
#     max_horizon=256,
#     normalize_inputs=True,    # TimesFM handles normalization internally
#     use_continuous_quantile_head=True,
#     force_flip_invariance=True,
#     infer_is_positive=False,  # set True only if your Y is strictly >= 0
#     fix_quantile_crossing=True,
# )
# model.compile(cfg)

# # 1) Your series: df indexed by datetime, column 'Y'
# # df = df.set_index("datetime").sort_index()
# y = df["Y"].astype("float32")

# # 2) Split and block settings
# split_ts  = pd.Timestamp("2004-11-01")  # start of test
# block_len = 5                           # use 3 if you want 3-step horizon

# test_y  = y.loc[y.index >= split_ts]
# test_idx = test_y.index

# # Build block start *row* indices over the test range (last-N-rows style)
# starts = np.arange(0, len(test_idx), block_len)
# block_starts = [test_idx[s] for s in starts]

# # 3) Build per-block contexts = all history strictly before each block start,
# #    truncated to the last cfg.max_context points
# def build_context(series: pd.Series, block_start_ts, max_context: int):
#     # everything up to (but excluding) the block start timestamp
#     end_loc = series.index.get_loc(block_start_ts)
#     ctx = series.iloc[:end_loc].to_numpy(dtype="float32")
#     if ctx.size == 0:
#         raise ValueError("Empty context before the first test timestamp.")
#     if ctx.size > max_context:
#         ctx = ctx[-max_context:]
#     return ctx

# contexts = [build_context(y, t0, cfg.max_context) for t0 in block_starts]

# print("4) Forecast all blocks at once (direct horizon = block_len)")
# with torch.inference_mode():
#     point_fcst, quantile_fcst = model.forecast(horizon=block_len, inputs=contexts)
#     # point_fcst.shape == (n_blocks, block_len)
#     # quantile_fcst.shape == (n_blocks, block_len, 10)  # mean, then q10..q90

# print("5) Stitch predictions back to timestamps and evaluate")
# preds, truths, times = [], [], []
# for b, t0 in enumerate(block_starts):
#     print(b,t0)
#     idx_block = test_idx[starts[b] : starts[b] + block_len]      # these are the next block_len rows
#     gt = test_y.loc[idx_block].to_numpy(dtype="float32")
#     ph = len(gt)                                                # last block may be shorter
#     preds.extend(point_fcst[b, :ph].tolist())
#     truths.extend(gt.tolist())
#     times.extend(idx_block[:ph].tolist())

# results = pd.DataFrame({"datetime": times, "y_true": truths, "y_pred": preds}).set_index("datetime")
# rmse = mean_squared_error(results["y_true"], results["y_pred"])
# print(f"RMSE (recursive with Y + covariate lags): {rmse:.3f}")
# rmse = mean_squared_error(results["y_true"], results["y_pred"])
# print(f"Device: {device}")
# print(f"RMSE (TimesFM, {block_len}-step direct): {rmse:.3f}")

In [11]:
# rmse = mean_squared_error(results["y_true"], results["y_pred"])
# print(f"Device: {device}")
# print(f"RMSE (TimesFM, {block_len}-step direct): {rmse:.3f}")


In [ ]:
import numpy as np, pandas as pd
from sklearn.metrics import mean_squared_error
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split
from uni2ts.model.moirai2 import Moirai2Forecast, Moirai2Module

# 0) Make the index uniform (hourly) and fill gaps
freq = 'H'  # hourly
dfu = (
    df[['Y']]                       # AR-only
      .sort_index()
      .asfreq(freq)                 # put missing hours in as NaN
)
dfu['Y'] = dfu['Y'].ffill().bfill() # no peeking: forward-fill, then back-fill only if needed

# 1) Split & windowing (same as TabPFN/TimesFM)
split_ts  = pd.Timestamp("2004-11-01 00:00:00")
block_len = 5                       # set 3 if you want 3-step horizon
TEST      = len(dfu.loc[dfu.index >= split_ts])
N_WINDOWS = TEST // block_len       # non-overlapping blocks

# 2) GluonTS dataset (target only)
ds_ar = PandasDataset({"series_0": dfu['Y']}, freq=freq)

train_ar, test_template_ar = split(ds_ar, date=pd.Period(split_ts, freq=freq))
test_data_ar = test_template_ar.generate_instances(
    prediction_length=block_len,
    windows=N_WINDOWS,
    distance=block_len,             # 0,5,10,… row-based blocks
)

# # 3) Moirai-2 small
# CTX, BSZ = 1000, 32
# model_ar = Moirai2Forecast(
#     module=Moirai2Module.from_pretrained("Salesforce/moirai-2.0-R-small"),
#     prediction_length=block_len,
#     context_length=CTX,
#     target_dim=1,
#     feat_dynamic_real_dim=0,
#     past_feat_dynamic_real_dim=0,
# )
# predictor_ar = model_ar.create_predictor(batch_size=BSZ)

# # 4) Forecast & evaluate
# forecasts_ar = list(predictor_ar.predict(test_data_ar.input))
# labels_ar    = list(test_data_ar.label)

# y_pred_ar = np.concatenate([f.mean for f in forecasts_ar])
# y_true_ar = np.concatenate([np.asarray(l["target"]) for l in labels_ar])

# rmse_ar = mean_squared_error(y_true_ar, y_pred_ar)
# print(f"RMSE (Moirai2 AR-only, {block_len}-step blocks): {rmse_ar:.3f}")


/Users/kilianrunnwerth/Desktop/Python/Kaggel/.conda/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/Users/kilianrunnwerth/Desktop/Python/Kaggel/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/kilianrunnwerth/Desktop/Python/Kaggel/.conda/lib/python3.11/site-packages/lightning/fabric/__init__.py:41: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
/var/folders/q9/d545ygdx5rj6lg_00gpykppw0000gn/T/ipykernel_64543/2448691764.py:12: FutureWarning: 'H' is deprecated and will be rem

: 

In [ ]:
import numpy as np, pandas as pd
from sklearn.metrics import mean_squared_error
from gluonts.dataset.common import ListDataset
from uni2ts.model.moirai2 import Moirai2Forecast, Moirai2Module

# 1) Canonical hourly data so all models compare apples-to-apples
freq = "H"
df_canon = df[["Y","T","RH","AH"]].sort_index().asfreq(freq)
df_canon["Y"] = df_canon["Y"].ffill()  # no look-ahead for target
for c in ["T","RH","AH"]:
    df_canon[c] = df_canon[c].interpolate(method="time").ffill().bfill()

# 2) Split + blocks (exactly like your other models)
split_ts  = pd.Timestamp("2004-11-01 00:00:00")
block_len = 5  # set 3 if you want 3-step horizon

test_idx = df_canon.index[df_canon.index >= split_ts]
starts   = np.arange(0, len(test_idx), block_len)

# Helper: build a single-entry GluonTS dataset for a block's context
def make_entry(y_series, end_ts, ctx, freq, t=None, rh=None, ah=None):
    """
    end_ts: the first timestamp of the block (we forecast from just before this)
    ctx:    how many last points to keep as context
    """
    pos = y_series.index.get_loc(end_ts)
    if isinstance(pos, slice):  # guard against duplicate indices
        pos = pos.start
    # history strictly before the block start
    if pos == 0:
        return None
    y_hist = y_series.values[:pos].astype("float32")
    idx_hist = y_series.index[:pos]
    # clip to last ctx points
    if len(y_hist) > ctx:
        y_hist  = y_hist[-ctx:]
        idx_hist = idx_hist[-ctx:]
    entry = {
        "start": idx_hist[0],
        "target": y_hist,
    }
    # add past covariates if provided (shape: F x time)
    if t is not None and rh is not None and ah is not None:
        T  = t.values[:pos].astype("float32")
        RH = rh.values[:pos].astype("float32")
        AH = ah.values[:pos].astype("float32")
        if len(T) > ctx:
            T, RH, AH = T[-ctx:], RH[-ctx:], AH[-ctx:]
        entry["past_feat_dynamic_real"] = [T, RH, AH]
    return ListDataset([entry], freq=freq)

# Keep memory modest first; raise later if stable
CTX, BSZ = 500, 4

model_ar = Moirai2Forecast(
    module=Moirai2Module.from_pretrained("Salesforce/moirai-2.0-R-small"),
    prediction_length=block_len,
    context_length=CTX,
    target_dim=1,
    feat_dynamic_real_dim=0,
    past_feat_dynamic_real_dim=0,
)
predictor_ar = model_ar.create_predictor(batch_size=BSZ)
try: predictor_ar = predictor_ar.to("cpu")  # avoids MPS/CUDA quirks
except: pass

sse, count, produced = 0.0, 0, 0
for b in range(len(starts)):
    start_ts = test_idx[starts[b]]
    ds_one   = make_entry(df_canon["Y"], end_ts=start_ts, ctx=CTX, freq=freq)
    if ds_one is None:
        continue

    # One forecast for this block
    f = next(predictor_ar.predict(ds_one))      # generator with 1 item
    y_hat  = f.quantile(0.5).astype("float32")  # explicit median, shape (block_len,)

    # Ground truth for the block
    idx_block = test_idx[starts[b] : starts[b] + block_len]
    y_true = df_canon["Y"].loc[idx_block].to_numpy(dtype="float32")

    ph = min(len(y_hat), len(y_true))           # tail-safe
    diff = y_hat[:ph] - y_true[:ph]
    sse  += float(np.dot(diff, diff))
    count += ph
    produced += 1

mse_ar  = sse / count
rmse_ar = (sse / count) ** 0.5
print(f"MSE  (Moirai2 AR-only, {block_len}-step blocks): {mse_ar:.3f}")
print(f"RMSE (Moirai2 AR-only, {block_len}-step blocks): {rmse_ar:.3f}")


/var/folders/q9/d545ygdx5rj6lg_00gpykppw0000gn/T/ipykernel_64543/3009199395.py:8: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_canon = df[["Y","T","RH","AH"]].sort_index().asfreq(freq)


In [ ]:
# --- Moirai-2 with past covariates (T, RH, AH) ---
CTX, BSZ = 500, 4  # keep same as AR for fair comparison

model_cov = Moirai2Forecast(
    module=Moirai2Module.from_pretrained("Salesforce/moirai-2.0-R-small"),
    prediction_length=block_len,
    context_length=CTX,
    target_dim=1,
    feat_dynamic_real_dim=0,      # no future-known features
    past_feat_dynamic_real_dim=3, # T, RH, AH
)
predictor_cov = model_cov.create_predictor(batch_size=BSZ)
try: predictor_cov = predictor_cov.to("cpu")
except: pass

sse, count, produced = 0.0, 0, 0
for b in range(len(starts)):
    start_ts = test_idx[starts[b]]
    ds_one = make_entry(
        df_canon["Y"], end_ts=start_ts, ctx=CTX, freq=freq,
        t=df_canon["T"], rh=df_canon["RH"], ah=df_canon["AH"]  # <-- pass covariates here
    )
    if ds_one is None:
        continue

    f = next(predictor_cov.predict(ds_one))
    y_hat = f.quantile(0.5).astype("float32")  # median as point forecast

    idx_block = test_idx[starts[b] : starts[b] + block_len]
    y_true = df_canon["Y"].loc[idx_block].to_numpy(dtype="float32")

    ph = min(len(y_hat), len(y_true))
    diff = y_hat[:ph] - y_true[:ph]
    sse += float(np.dot(diff, diff))
    count += ph
    produced += 1

mse_cov  = sse / count
rmse_cov = (sse / count) ** 0.5
print(f"MSE  (Moirai2 + past covariates, {block_len}-step blocks): {mse_cov:.3f}")
print(f"RMSE (Moirai2 + past covariates, {block_len}-step blocks): {rmse_cov:.3f}")


MSE  (Moirai2 + past covariates, 5-step blocks): 1.333
RMSE (Moirai2 + past covariates, 5-step blocks): 1.154


## Flowstate

In [1]:
import numpy as np
import pandas as pd
import torch
from tsfm_public import FlowStateForPrediction
from sklearn.metrics import mean_squared_error

# ---------- 0) Canonical hourly data (so all models compare apples-to-apples) ----------
freq = "H"
df_canon = df[["Y","T","RH","AH"]].sort_index().asfreq(freq)

# Causal fills (no look-ahead)
df_canon["Y"] = df_canon["Y"].ffill()
for c in ["T","RH","AH"]:
    df_canon[c] = df_canon[c].ffill()

# ---------- 1) Shared split & blocks ----------
split_ts  = pd.Timestamp("2004-11-01 00:00:00")
block_len = 5   # set 3 if that's your official horizon

test_idx = df_canon.index[df_canon.index >= split_ts]
starts   = np.arange(0, len(test_idx), block_len)  # 0,5,10,...

# ---------- 2) Load FlowState predictor ----------
device = "cuda" if torch.cuda.is_available() else "cpu"
predictor = FlowStateForPrediction.from_pretrained("ibm-research/flowstate").to(device)

# Keep this equal to your TabPFN lag count for fairness.
CTX_LEN = 72
BATCH   = 32
SCALE   = 0.25
Q_MED   = 0.5  # median

# ---------- 3) Helper to build batched contexts (AR-only: 1 channel) ----------
def build_batch_contexts_y(y: pd.Series, start_ts_list, ctx_len, device):
    """
    Returns (ctx, kept_starts):
      ctx: FloatTensor [ctx_len, batch, 1] (batch_first=False)
      kept_starts: the subset of start_ts that had enough history
    """
    yv  = y.to_numpy(dtype="float32")
    idx = y.index
    batches = []
    kept = []
    for t0 in start_ts_list:
        pos = idx.get_loc(t0)
        if isinstance(pos, slice):
            pos = pos.start
        if pos < ctx_len:
            continue
        window = yv[pos - ctx_len : pos]          # strictly before t0
        batches.append(window[:, None])           # (ctx_len, 1)
        kept.append(t0)
    if not batches:
        return None, []
    X = np.stack(batches, axis=1)                 # (ctx_len, batch, 1)
    return torch.from_numpy(X).to(device), kept

# ---------- 4a) DIRECT multi-horizon (faster; FlowState predicts block_len in one call) ----------
def flowstate_direct(y: pd.Series, test_idx, starts, ctx_len, block_len, batch, device):
    preds, trues, times = [], [], []
    for i in range(0, len(starts), batch):
        start_slice = [test_idx[s] for s in starts[i:i+batch] if s < len(test_idx)]
        if not start_slice:
            continue
        ctx, kept = build_batch_contexts_y(y, start_slice, ctx_len, device)
        if ctx is None:
            continue

        with torch.inference_mode():
            out = predictor(
                ctx, scale_factor=SCALE, prediction_length=block_len, batch_first=False
            )
        po = out.prediction_outputs  # (batch, num_quantiles, horizon, 1)

        # choose the closest available quantile to 0.5 (median)
        if hasattr(out, "quantile_values"):
            qs = torch.tensor(out.quantile_values, device=po.device)
            q_idx = int(torch.argmin(torch.abs(qs - Q_MED)).item())
        else:
            q_idx = po.shape[1] // 2

        y_hat = po[:, q_idx, :block_len, 0].detach().cpu().numpy()

        # stitch per block
        for j, t0 in enumerate(kept):
            start_row = int(np.where(test_idx == t0)[0][0])
            idx_block = test_idx[start_row : start_row + block_len]
            y_true = y.loc[idx_block].to_numpy(dtype="float32")
            ph = len(y_true)
            preds.extend(y_hat[j, :ph].tolist())
            trues.extend(y_true.tolist())
            times.extend(idx_block.tolist())

    res = pd.DataFrame({"datetime": times, "y_true": trues, "y_pred": preds}).set_index("datetime")
    mse  = mean_squared_error(res["y_true"], res["y_pred"])
    rmse = mean_squared_error(res["y_true"], res["y_pred"])
    return res, mse, rmse

# ---------- 4b) RECURSIVE 1-step × block_len (mirrors your TabPFN/linear recursion) ----------
def flowstate_recursive(y: pd.Series, test_idx, starts, ctx_len, block_len, batch, device):
    preds, trues, times = [], [], []
    # work in batches of blocks
    for i in range(0, len(starts), batch):
        start_slice = [test_idx[s] for s in starts[i:i+batch] if s < len(test_idx)]
        if not start_slice:
            continue

        # init contexts for these blocks
        ctx, kept = build_batch_contexts_y(y, start_slice, ctx_len, device)
        if ctx is None:
            continue

        # we will mutate a copy of the context for recursive stepping
        ctx_work = ctx.clone()

        # predict 1 step, append to context, repeat block_len times
        batch_kept = ctx_work.shape[1]
        step_preds = np.zeros((batch_kept, block_len), dtype="float32")

        for h in range(block_len):
            with torch.inference_mode():
                out = predictor(
                    ctx_work, scale_factor=SCALE, prediction_length=1, batch_first=False
                )
            po = out.prediction_outputs  # (batch, num_quantiles, 1, 1)
            if hasattr(out, "quantile_values"):
                qs = torch.tensor(out.quantile_values, device=po.device)
                q_idx = int(torch.argmin(torch.abs(qs - Q_MED)).item())
            else:
                q_idx = po.shape[1] // 2

            y_hat_h = po[:, q_idx, 0, 0]              # (batch,)
            step_preds[:, h] = y_hat_h.detach().cpu().numpy()

            # shift: drop oldest, append current prediction
            ctx_work = torch.cat([ctx_work[1:], y_hat_h.view(1, batch_kept, 1)], dim=0)

        # stitch per block
        for j, t0 in enumerate(kept):
            start_row = int(np.where(test_idx == t0)[0][0])
            idx_block = test_idx[start_row : start_row + block_len]
            y_true = y.loc[idx_block].to_numpy(dtype="float32")
            ph = len(y_true)
            preds.extend(step_preds[j, :ph].tolist())
            trues.extend(y_true.tolist())
            times.extend(idx_block.tolist())

    res = pd.DataFrame({"datetime": times, "y_true": trues, "y_pred": preds}).set_index("datetime")
    mse  = mean_squared_error(res["y_true"], res["y_pred"])
    rmse = mean_squared_error(res["y_true"], res["y_pred"])
    return res, mse, rmse

# ---------- 5) Run both modes ----------
results_dir, mse_dir, rmse_dir = flowstate_direct(
    df_canon["Y"], test_idx, starts, CTX_LEN, block_len, BATCH, device
)
print(f"[FlowState AR-only | DIRECT]  blocks={len(results_dir)//block_len}  MSE={mse_dir:.3f}  RMSE={rmse_dir:.3f}")

results_rec, mse_rec, rmse_rec = flowstate_recursive(
    df_canon["Y"], test_idx, starts, CTX_LEN, block_len, BATCH, device
)
print(f"[FlowState AR-only | RECURSIVE]  blocks={len(results_rec)//block_len}  MSE={mse_rec:.3f}  RMSE={rmse_rec:.3f}")


/Users/kilianrunnwerth/Desktop/Python/Kaggel/.conda/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/Users/kilianrunnwerth/Desktop/Python/Kaggel/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'df' is not defined

In [ ]:
# ===== Chronos / Chronos-Bolt — AR-only, comparable to your TabPFN & linear =====
# pip install -U chronos-forecasting

import numpy as np
import pandas as pd
import torch
from sklearn.metrics import mean_squared_error
from chronos import BaseChronosPipeline  # Chronos-T5 or Chronos-Bolt

# ---------- 0) Canonical hourly data (apples-to-apples across models) ----------
freq = "H"
df_canon = df[["Y","T","RH","AH"]].sort_index().asfreq(freq)

# causal fills (no future peek)
df_canon["Y"] = df_canon["Y"].ffill()
for c in ["T","RH","AH"]:
    df_canon[c] = df_canon[c].ffill()

# ---------- 1) Shared split & row-blocks ----------
split_ts  = pd.Timestamp("2004-11-01 00:00:00")
block_len = 5  # set 3 if you want a 3-step horizon

test_idx = df_canon.index[df_canon.index >= split_ts]
starts   = np.arange(0, len(test_idx), block_len)  # 0,5,10,...

# ---------- 2) Load a Chronos pipeline ----------
# Bolt is fast & memory-efficient: "amazon/chronos-bolt-small" or "amazon/chronos-bolt-base"
model_id   = "amazon/chronos-bolt-small"
device_map = "cuda" if torch.cuda.is_available() else "cpu"
dtype      = torch.bfloat16 if torch.cuda.is_available() else torch.float32

pipeline = BaseChronosPipeline.from_pretrained(
    model_id,
    device_map=device_map,
    torch_dtype=dtype,
)

# Keep Chronos context equal to your TabPFN lags for fairness
CTX_LEN = 72
BATCH   = 64  # drop to 16/32 if tight on memory

# ---------- 3) Helper: build list of contexts (one per block start) ----------
def build_context_list(y: pd.Series, start_ts_list, ctx_len):
    yv, idx = y.to_numpy(dtype="float32"), y.index
    contexts, kept = [], []
    for t0 in start_ts_list:
        pos = idx.get_loc(t0)
        if isinstance(pos, slice):  # guard against duplicate indices
            pos = pos.start
        if pos < ctx_len:
            continue
        contexts.append(torch.tensor(yv[pos-ctx_len:pos]))
        kept.append(t0)
    return contexts, kept

# ---------- 4a) DIRECT: predict block_len in one call (Chronos-Bolt native) ----------
def chronos_direct(y: pd.Series, test_idx, starts, ctx_len, block_len, batch):
    preds, trues, times = [], [], []

    for i in range(0, len(starts), batch):
        start_slice = [test_idx[s] for s in starts[i:i+batch] if s < len(test_idx)]
        if not start_slice:
            continue

        contexts, kept = build_context_list(y, start_slice, ctx_len)
        if not contexts:
            continue

        with torch.inference_mode():
            # predict_quantiles returns (quantiles, mean)
            # quantiles: [batch, pred_len, n_q]; mean: [batch, pred_len]
            quantiles, mean = pipeline.predict_quantiles(
                context=contexts,                # list of 1D tensors
                prediction_length=block_len,
                quantile_levels=[0.1, 0.5, 0.9],
            )

        y_hat = mean.numpy()  # (batch_kept, block_len) — use mean for MSE/RMSE

        # stitch back to timestamps
        for j, t0 in enumerate(kept):
            start_row = int(np.where(test_idx == t0)[0][0])
            idx_block = test_idx[start_row : start_row + block_len]
            y_true = y.loc[idx_block].to_numpy(dtype="float32")
            ph = len(y_true)
            preds.extend(y_hat[j, :ph].tolist())
            trues.extend(y_true.tolist())
            times.extend(idx_block.tolist())

    results = pd.DataFrame({"datetime": times, "y_true": trues, "y_pred": preds}).set_index("datetime")
    mse  = mean_squared_error(results["y_true"], results["y_pred"])
    rmse = mean_squared_error(results["y_true"], results["y_pred"])
    return results, mse, rmse

# ---------- 4b) RECURSIVE: 1-step × block_len (mirrors your TabPFN loop exactly) ----------
def chronos_recursive(y: pd.Series, test_idx, starts, ctx_len, block_len, batch):
    preds, trues, times = [], [], []

    for i in range(0, len(starts), batch):
        start_slice = [test_idx[s] for s in starts[i:i+batch] if s < len(test_idx)]
        if not start_slice:
            continue

        contexts, kept = build_context_list(y, start_slice, ctx_len)
        if not contexts:
            continue

        # mutable contexts (drop oldest, append prediction each step)
        run_ctx = [c.clone().to(torch.float32) for c in contexts]  # one (ctx_len,) per block

        step_preds = np.zeros((len(run_ctx), block_len), dtype="float32")

        for h in range(block_len):
            with torch.inference_mode():
                _, mean_h = pipeline.predict_quantiles(
                    context=run_ctx, prediction_length=1, quantile_levels=[0.5]
                )
            y_hat_h = mean_h[:, 0].numpy()  # (batch,)

            # update contexts (shift left + append)
            for j in range(len(run_ctx)):
                x = run_ctx[j].numpy()
                x = np.concatenate([x[1:], y_hat_h[j:j+1]]).astype("float32")
                run_ctx[j] = torch.from_numpy(x)

            step_preds[:, h] = y_hat_h

        # stitch predictions to timestamps
        for j, t0 in enumerate(kept):
            start_row = int(np.where(test_idx == t0)[0][0])
            idx_block = test_idx[start_row : start_row + block_len]
            y_true = y.loc[idx_block].to_numpy(dtype="float32")
            ph = len(y_true)
            preds.extend(step_preds[j, :ph].tolist())
            trues.extend(y_true.tolist())
            times.extend(idx_block.tolist())

    results = pd.DataFrame({"datetime": times, "y_true": trues, "y_pred": preds}).set_index("datetime")
    mse  = mean_squared_error(results["y_true"], results["y_pred"])
    rmse = mean_squared_error(results["y_true"], results["y_pred"])
    return results, mse, rmse

# ---------- 5) Run both modes ----------
results_dir, mse_dir, rmse_dir = chronos_direct(df_canon["Y"], test_idx, starts, CTX_LEN, block_len, BATCH)
print(f"[Chronos DIRECT]    blocks={len(results_dir)//block_len}  MSE={mse_dir:.3f}  RMSE={rmse_dir:.3f}")

results_rec, mse_rec, rmse_rec = chronos_recursive(df_canon["Y"], test_idx, starts, CTX_LEN, block_len, BATCH)
print(f"[Chronos RECURSIVE] blocks={len(results_rec)//block_len}  MSE={mse_rec:.3f}  RMSE={rmse_rec:.3f}")


/Users/kilianrunnwerth/Desktop/Python/Kaggel/.conda/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/Users/kilianrunnwerth/Desktop/Python/Kaggel/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/q9/d545ygdx5rj6lg_00gpykppw0000gn/T/ipykernel_64070/2136177935.py:12: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_canon = df[["Y","T","RH","AH"]].sort_index().asfreq(freq)


: 

Yinglong

In [ ]:
!pip install -U transformers xformers  # xformers optional

  Using cached transformers-4.56.2-py3-none-any.whl.metadata (40 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 5.9 MB/s  0:00:02m0:00:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached tokenizers-0.22.1-cp39-abi3-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached torch-2.8.0-cp311-none-macosx_11_0_arm64.whl.metadata (30 kB)
Using cached transformers-4.56.2-py3-none-any.whl (11.6 MB)
Using cached tokenizers-0.22.1-cp39-abi3-macosx_11_0_arm64.whl (2.9 MB)
Using cached torch-2.8.0-cp311-none-macosx_11_0_arm64.whl (73.6 MB)
  error: subprocess-exited-with-error
  
  × Building wheel for xformers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [213 lines of output]
      /private/var/folders/q9/d545ygdx5rj6lg_00gpykppw0000gn/T/pip-build-env-kzq5gl8a/overlay/lib/python3.11/site-packages/torch/_subclasses/functional_tensor.py:279: UserWarning: Faile

In [ ]:
# ===================== YingLong — AR-only, TabPFN-comparable =====================
# 
# If you hit flash-attn import issues, use CPU or the 6m model (see note below).  :contentReference[oaicite:1]{index=1}

import numpy as np
import pandas as pd
import torch
from sklearn.metrics import mean_squared_error
from transformers import AutoModelForCausalLM

# ---------- 0) Canonical hourly data (same as the other models) ----------
freq = "H"
df_canon = df[["Y","T","RH","AH"]].sort_index().asfreq(freq)
df_canon["Y"] = df_canon["Y"].ffill()
for c in ["T","RH","AH"]:
    df_canon[c] = df_canon[c].ffill()

# ---------- 1) Shared split & row-blocks ----------
split_ts  = pd.Timestamp("2004-11-01 00:00:00")
block_len = 5  # set 3 if you want a 3-step horizon
test_idx  = df_canon.index[df_canon.index >= split_ts]
starts    = np.arange(0, len(test_idx), block_len)  # 0,5,10,...

# ---------- 2) Load YingLong ----------
# Pick a size. Start with 6m for reliability; switch to 110m/300m if you have GPU headroom.
model_id = "qcw2333/YingLong_6m"  # alternatives: "qcw2333/YingLong_110m", "qcw2333/YingLong_300m"  :contentReference[oaicite:2]{index=2}
device   = "cuda" if torch.cuda.is_available() else "cpu"
dtype    = torch.bfloat16 if (device == "cuda") else torch.float32

# HF quickstart uses: AutoModelForCausalLM.from_pretrained(..., trust_remote_code=True).generate(future_token=...)  :contentReference[oaicite:3]{index=3}
model = AutoModelForCausalLM.from_pretrained(
    model_id, trust_remote_code=True, torch_dtype=dtype
).to(device)
model.eval()

# Keep context equal to your TabPFN lags for fairness (you can raise later)
CTX_LEN = 72
BATCH   = 64  # drop to 16/32 if memory is tight

# ---------- 3) Helpers ----------
def build_batch_contexts_y(y: pd.Series, start_ts_list, ctx_len):
    """
    Return (X, kept):
      X: FloatTensor [batch, ctx_len] of past Y strictly before each block start
      kept: the subset of timestamps that had >= ctx_len history
    """
    yv, idx = y.to_numpy(dtype="float32"), y.index
    rows, kept = [], []
    for t0 in start_ts_list:
        pos = idx.get_loc(t0)
        if isinstance(pos, slice): pos = pos.start
        if pos < ctx_len:  # not enough history
            continue
        rows.append(yv[pos-ctx_len:pos])
        kept.append(t0)
    if not rows:
        return None, []
    X = torch.tensor(np.stack(rows, axis=0), dtype=torch.float32, device=device)
    # cast up to model dtype on GPU
    return (X.to(dtype) if device == "cuda" else X), kept

@torch.inference_mode()
def yinglong_direct(y: pd.Series, test_idx, starts, ctx_len, block_len, batch):
    preds, trues, times = [], [], []
    for i in range(0, len(starts), batch):
        start_slice = [test_idx[s] for s in starts[i:i+batch] if s < len(test_idx)]
        if not start_slice: continue
        X, kept = build_batch_contexts_y(y, start_slice, ctx_len)
        if X is None: continue

        # GENERATE block_len steps in one shot (DIRECT)
        out = model.generate(X, future_token=block_len)  # -> [batch, block_len]  :contentReference[oaicite:4]{index=4}
        y_hat = out.detach().to(torch.float32).cpu().numpy()

        for j, t0 in enumerate(kept):
            start_row = int(np.where(test_idx == t0)[0][0])
            idx_block = test_idx[start_row : start_row + block_len]
            y_true = y.loc[idx_block].to_numpy(dtype="float32")
            ph = len(y_true)
            preds.extend(y_hat[j, :ph].tolist())
            trues.extend(y_true.tolist())
            times.extend(idx_block.tolist())

    res  = pd.DataFrame({"datetime": times, "y_true": trues, "y_pred": preds}).set_index("datetime")
    mse  = mean_squared_error(res["y_true"], res["y_pred"])
    rmse = mean_squared_error(res["y_true"], res["y_pred"], squared=False)
    return res, mse, rmse

@torch.inference_mode()
def yinglong_recursive(y: pd.Series, test_idx, starts, ctx_len, block_len, batch):
    preds, trues, times = [], [], []
    for i in range(0, len(starts), batch):
        start_slice = [test_idx[s] for s in starts[i:i+batch] if s < len(test_idx)]
        if not start_slice: continue
        X, kept = build_batch_contexts_y(y, start_slice, ctx_len)
        if X is None: continue

        # RECURSIVE: 1-step at a time, shift window, append prediction
        B = X.shape[0]
        step_preds = np.zeros((B, block_len), dtype="float32")
        ctx_work = X.clone()

        for h in range(block_len):
            out = model.generate(ctx_work, future_token=1)        # [B, 1]
            y_hat_h = out.squeeze(-1) if out.ndim == 3 else out   # be tolerant of shape
            y_hat_h = y_hat_h.view(B, -1)[:, 0]                   # [B]
            step_preds[:, h] = y_hat_h.detach().to(torch.float32).cpu().numpy()
            # update rolling context (drop oldest, append prediction)
            ctx_work = torch.cat([ctx_work[:, 1:], y_hat_h[:, None].to(ctx_work.dtype)], dim=1)

        for j, t0 in enumerate(kept):
            start_row = int(np.where(test_idx == t0)[0][0])
            idx_block = test_idx[start_row : start_row + block_len]
            y_true = y.loc[idx_block].to_numpy(dtype="float32")
            ph = len(y_true)
            preds.extend(step_preds[j, :ph].tolist())
            trues.extend(y_true.tolist())
            times.extend(idx_block.tolist())

    res  = pd.DataFrame({"datetime": times, "y_true": trues, "y_pred": preds}).set_index("datetime")
    mse  = mean_squared_error(res["y_true"], res["y_pred"])
    rmse = mean_squared_error(res["y_true"], res["y_pred"], squared=False)
    return res, mse, rmse

# ---------- 4) Run both modes ----------
results_dir, mse_dir, rmse_dir = yinglong_direct(df_canon["Y"], test_idx, starts, CTX_LEN, block_len, BATCH)
print(f"[YingLong DIRECT]    blocks={len(results_dir)//block_len}  MSE={mse_dir:.3f}  RMSE={rmse_dir:.3f}")

results_rec, mse_rec, rmse_rec = yinglong_recursive(df_canon["Y"], test_idx, starts, CTX_LEN, block_len, BATCH)
print(f"[YingLong RECURSIVE] blocks={len(results_rec)//block_len}  MSE={mse_rec:.3f}  RMSE={rmse_rec:.3f}")


/var/folders/q9/d545ygdx5rj6lg_00gpykppw0000gn/T/ipykernel_48738/768323795.py:13: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_canon = df[["Y","T","RH","AH"]].sort_index().asfreq(freq)
A new version of the following files was downloaded from https://huggingface.co/qcw2333/YingLong_6m:
- model_config.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
Encountered exception while importing flash_attn: No module named 'flash_attn'
Encountered exception while importing rotary_emb: No module named 'rotary_emb'
Encountered exception while importing dropout_layer_norm: No module named 'dropout_layer_norm'
Encountered exception while importing xformers: No module named 'xformers'


ImportError: This modeling file requires the following packages that were not found in your environment: flash_attn, rotary_emb, dropout_layer_norm, xformers. Run `pip install flash_attn rotary_emb dropout_layer_norm xformers`

In [6]:
from tsfm_public import FlowStateForPrediction
import torch
ädevice= 'cuda'
predictor = FlowStateForPrediction.from_pretrained("ibm-granite/granite-timeseries-flowstate-r1")
time_series = torch.randn((2048, 32, 1)) # context, batch, n_ch
forecast = predictor(time_series, scale_factor=0.25, prediction_length=960, batch_first=False)
print(forecast.prediction_outputs.shape) # torch.Size([32, 9, 48, 1]) (batch, quantiles, forecast_length, n_ch)


INFO:p-8346:t-8754634944:modeling_flowstate.py:__init__:Number of encoder parameters: 7885.8240000000005k
INFO:p-8346:t-8754634944:modeling_flowstate.py:__init__:Number of dencoder parameters: 1181.952k (14.99%)


torch.Size([32, 9, 960, 1])
